In [2]:
#!pip install selenium
#!pip install pandas
#!pip install openpyxl

Importamos el módulo **sys**

In [ ]:
import sys

Definimos la ruta base de búsqueda de paquetes y módulos

In [ ]:
sys.path.append('../../')

Importamos los módulos necesarios de la librería **Selenium** de Python

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin

import time
import datetime
import random
import os

import pandas as pd

Importamos los módulos **Data** y **Ficheros** de nuestra librería

In [ ]:
import curso.api.Data as Data
import curso.api.Ficheros as Files
import curso.api.Cadena as Cadena
import curso.api.Texto as Texto
import curso.api.Input as Input

Estructura del esquema del archivo '**provincias.data**'

In [ ]:
SCHEMA_PROVINCIA = {}
SCHEMA_PROVINCIA['id'] = Input.to_int
SCHEMA_PROVINCIA['provincia'] = Input.to_str
SCHEMA_PROVINCIA['id_ccaa'] = Input.to_int
SCHEMA_PROVINCIA['nombre'] = Input.to_str
SCHEMA_PROVINCIA['formato'] = Input.to_int

Estructura del esquema del archivo de **poblaciones**

In [ ]:
SCHEMA_POBLACION = {}
SCHEMA_POBLACION['id'] = Input.to_int
SCHEMA_POBLACION['id_ccaa'] = Input.to_int
SCHEMA_POBLACION['id_provincia'] = Input.to_int
SCHEMA_POBLACION['poblacion'] = Input.to_str
SCHEMA_POBLACION['nombre'] = Input.to_str
SCHEMA_POBLACION['url'] = Input.to_str

Estructura del esquema del archivo de **temperaturas**

In [ ]:
SCHEMA_DATA = {}
SCHEMA_DATA['fecha'] = Input.to_date
SCHEMA_DATA['id_ccaa'] = Input.to_int
SCHEMA_DATA['id_provincia'] = Input.to_int
SCHEMA_DATA['id_poblacion'] = Input.to_int
SCHEMA_DATA['hora'] = Input.to_str
SCHEMA_DATA['temperatura'] = Input.to_float
SCHEMA_DATA['viento'] = Input.to_float

Ruta base de los datasets de poblaciones

In [4]:
RUTA_BASE_ELTIEMPO = 'c:/datasets/data/eltiempo/'
RUTA_DATASETS_POBLACIONES = RUTA_BASE_ELTIEMPO + 'poblaciones/'

Nombres de archivos de los datasets

In [ ]:
FILE_DATA_PROVINCIAS = RUTA_BASE_ELTIEMPO + 'provincias.data'
FILE_DATA_POBLACIONES = RUTA_BASE_ELTIEMPO + 'poblaciones.data'
FILE_DATA_TEMPERATURAS = RUTA_BASE_ELTIEMPO + 'temperaturas.data'

Cargamos el dataset '**provincias.data**'

In [ ]:
dataset_provincias = Data.data_record(FILE_DATA_PROVINCIAS, header=True, schema=SCHEMA_PROVINCIA)

Mostramos el contenido del dataset **dataset_provincias**

In [ ]:
dataset_provincias

Dirección del sitio web para realizar Web Scraping

In [ ]:
URL_SITE_EL_TIEMPO = 'https://www.eltiempo.es/'

Funciones básicas para el funcionamiento de la aplicación Web Scraping

In [ ]:
# función que recibe una colección tipo diccionario, agrega un nuevo campo 'url' a la colección y 
# devuelve la colección modificada
def to_url_eltiempo(data):
    # https://www.eltiempo.es/cantabria - formato 0
    # https://www.eltiempo.es/en-provincia-huelva - formato 1
    # https://www.eltiempo.es/melilla.html - formato 2
    url = ''
    if data['formato'] == 1:
        url = 'en-provincia-' + data['nombre']
    elif data['formato'] == 2:
        url = data['nombre'] + '.html'
    else:
        url = data['nombre']
    data['url'] = URL_SITE_EL_TIEMPO + url
    return data

# función que recibe una colección de tipo diccionario y devuelve los valores de la colección como una cadena de texto
def dict_to_string(data):
    return ';'.join(str(value) for value in data.values())

# función que devuelve todas las poblaciones del sitio web 'eltiempo.es'
def extraer_poblaciones_eltiempo(url_site, web_provincias):
    driver = webdriver.Chrome()
    try:
        driver.get(url_site)
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a._10qqh8uq'))).click()
        time.sleep(3)
        dominio = url_site.split('/')[2]
        id_poblacion = 0
        header = ';'.join(SCHEMA_POBLACION.keys())
        for provincia in web_provincias:
            if provincia['formato'] == 2:
                continue
            print('Extrayendo de => ' + provincia['url'])
            driver.get(provincia['url'])
            time.sleep(5)
            scroll_origin = ScrollOrigin.from_viewport(10, 10)
            ActionChains(driver).scroll_from_origin(scroll_origin, 0, 1200).perform()
            ActionChains(driver).scroll_from_origin(scroll_origin, 0, 100).perform()
            data_poblaciones = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'article.modules.m_ccaaprovince.m39')))
            links = [(str(tag.get_attribute('href')), tag.text) for tag in data_poblaciones.find_elements(By.TAG_NAME, 'a')]
            poblaciones = []
            for url, text in links:
                id_poblacion += 1
                poblacion = {}
                poblacion['id'] = id_poblacion
                poblacion['id_ccaa'] = provincia['id_ccaa']
                poblacion['id_provincia'] = provincia['id']
                poblacion['poblacion'] = text
                poblacion['nombre'] = url.split('/')[-1].split('.')[0]
                poblacion['url'] = url
                poblaciones.append(poblacion)
            poblaciones = map(dict_to_string, poblaciones)
            Files.save_file_list(RUTA_DATASETS_POBLACIONES + provincia['nombre'] + '.data', poblaciones, header=header)
    finally:
        driver.quit()

Creamos la colección '**web_provincias**' utilizando el dataset ***dataset_provincias***

In [ ]:
web_provincias = list(map(to_url_eltiempo, dataset_provincias))

Mostramos el contenido de la colección '**web_provincias**'

In [ ]:
for p in web_provincias:
    print(p)
#Files.save_file_list('c:/datasets/data/eltiempo/provincias.links', web_provincias)

Web Scraping para obtener todas las poblaciones del sitio web **eltiempo.es** y crear *datasets de poblaciones* con las urls

In [ ]:
extraer_poblaciones_eltiempo(URL_SITE_EL_TIEMPO, web_provincias)

Función que devuelve la unión de todos los datasets de poblaciones generados

In [ ]:
def get_by_id(id, collection):
    return list(filter(lambda c: c['id'] == id, collection))[0]
    
def merge(directory):
    data = []
    for file in Files.dir_files(directory):
        data += Data.data_record(file, header=True, schema=SCHEMA_POBLACION)
    return data

Realizamos la unión de todos los datasets de poblaciones generados

In [ ]:
dataset_poblaciones = merge(RUTA_DATASETS_POBLACIONES)
print(len(dataset_poblaciones))

Guardamos el dataset '**dataset_poblaciones**' que contiene todas las poblaciones

In [ ]:
dataset_poblaciones = map(dict_to_string, dataset_poblaciones)
header = ';'.join(SCHEMA_POBLACION.keys())
Files.save_file_list(FILE_DATA_POBLACIONES, dataset_poblaciones, header=header)

Limpiamos el dataset '**dataset_poblaciones**' de urls repetidas

In [ ]:
def remove_duplicate(collection, field):
    value = ''
    data = []
    for item in collection:
        data_field = Texto.to_abc(item[field])
        if data_field == value:
            continue
        value = data_field
        data.append(item)
    return data

Cargamos de nuevo el dataset '**dataset_poblaciones**'

In [ ]:
dataset_poblaciones = Data.data_record(FILE_DATA_POBLACIONES, header=True, schema=SCHEMA_POBLACION)

Eliminamos los enlaces duplicados que puedan aparecer

In [ ]:
print(len(dataset_poblaciones))
dataset_poblaciones = remove_duplicate(dataset_poblaciones, 'nombre')
print(len(dataset_poblaciones))

Guardamos el dataset '**dataset_poblaciones**'

In [ ]:
header = ';'.join(SCHEMA_POBLACION.keys())
dataset_poblaciones = map(dict_to_string, dataset_poblaciones)
Files.save_file_list(FILE_DATA_POBLACIONES, dataset_poblaciones, header=header)

In [ ]:
def temperaturas_diarias(driver, poblacion):
    temperaturas = []
    for li in driver.find_element(By.ID, 'meteograma').find_elements(By.TAG_NAME, 'ul')[1].find_elements(By.TAG_NAME, 'li'):
        datos = Cadena.trim(li.text).split(' ')
        data = {}
        data['fecha'] = datetime.date.today()
        data['id_ccaa'] = Input.to_int(poblacion['id_ccaa'])
        data['id_provincia'] = Input.to_int(poblacion['id_provincia'])
        data['id_poblacion'] = Input.to_int(poblacion['id'])
        data['hora'] = datos[0]
        data['temperatura'] = Input.to_float(datos[1][:-1])
        data['viento'] = Input.to_float(datos[5])
        temperaturas.append(data)
    return temperaturas

In [ ]:
dataset_provincias = Data.data_record(FILE_DATA_PROVINCIAS, header=True, schema=SCHEMA_PROVINCIA)
dataset_poblaciones = Data.data_record(FILE_DATA_POBLACIONES, header=True, schema=SCHEMA_POBLACION)

id_provincia = 11
provincia = get_by_id(id_provincia, dataset_provincias)
driver = webdriver.Chrome()

try:
    driver.get(URL_SITE_EL_TIEMPO)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a._10qqh8uq'))).click()
    time.sleep(3)
    if not os.path.exists(FILE_DATA_TEMPERATURAS):
        with open(FILE_DATA_TEMPERATURAS, mode='w', encoding='utf-8') as file:
            file.write(';'.join(SCHEMA_DATA.keys()) + '\n')
    dataset = list(filter(lambda poblacion: poblacion['id_provincia'] == provincia['id'], dataset_poblaciones))
    for n, poblacion in enumerate(dataset):
        if n == 10:
            break
        print(f"{provincia['provincia']} [{n + 1}:{len(dataset)}] '{poblacion['poblacion']}' => {poblacion['url']}")
        driver.get(poblacion['url'] + '?v=por_hora')
        time.sleep(5)
        temperaturas = temperaturas_diarias(driver, poblacion)
        dataset_temperaturas = list(map(dict_to_string, temperaturas))
        Files.save_file_list(FILE_DATA_TEMPERATURAS, dataset_temperaturas, mode='a')
finally:
    driver.quit()

In [ ]:
dataset_temperaturas = Data.data_record(FILE_DATA_TEMPERATURAS, header=True, schema=SCHEMA_DATA)

In [ ]:
dataset_temperaturas

Cargamos el dataset de temperaturas en un DataFrame de **pandas**

In [ ]:
df = pd.DataFrame(dataset_temperaturas)
df

Guardamos el dataframe de pandas como un archivo **json**

In [ ]:
df.to_json(FILE_DATA_TEMPERATURAS + '.json', date_format='iso', date_unit='s')

Creamos el dataframe pandas 'df_json' utilizando el archivo json creado anteriormente

In [ ]:
df_json = pd.read_json(FILE_DATA_TEMPERATURAS + '.json')
df_json['fecha'] = pd.to_datetime(df_json['fecha'])
df_json

Guardamos el dataframe de pandas como un archivo **Excel**

In [ ]:
df.to_excel(FILE_DATA_TEMPERATURAS + '.xlsx')

Creamos el dataframe pandas 'df_excel' utilizando el archivo Excel creado anteriormente

In [ ]:
df_excel = pd.read_excel(FILE_DATA_TEMPERATURAS + '.xlsx')
df_excel